Headers

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from scipy.ndimage import zoom
from keras.layers import Conv2D
from keras.layers import UpSampling2D
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import RepeatVector
from keras.layers import Reshape
from keras.utils import plot_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Add
from tensorflow.keras.layers import BatchNormalization
import math
import tifffile
import os
from pickle import load
import numpy as np
import imageio
from matplotlib import pyplot as plt
from keras.layers import Conv2DTranspose

In [2]:
cd 

C:\Users\otiwa


Preprocessing data

In [3]:
#function to slice the image
def slicing(lr_train_images, subset_size):
    '''a simple but very effective function for slicing the input in to (128 X 128) and the ground truth into (256 X 256)'''

    # Calculate the number of subsets
    num_subsets = np.array(lr_train_images.shape[:2]) // subset_size

    # Initialize an empty list to store the subsets
    subsets = []

    # Loop over the number of subsets in rows and columns
    for i in range(num_subsets[0]):
        for j in range(num_subsets[1]):
            # Calculate the starting indices of the subset
            start_row = i * subset_size
            start_col = j * subset_size

            # Extract the subset from the image
            subset = lr_train_images[start_row:start_row+subset_size, start_col:start_col+subset_size]

            # Append the subset to the list
            subsets.append(subset)

    # Convert the list of subsets to a numpy array
    subsets_array = np.array(subsets)

    return subsets_array

In [4]:
#function to preprocess
def preprocess_image(image, no, count):
    '''The preprocessing step involves normalisation between -1 to 1 and slicing the image'''
    
    count += 1
    image_np = np.array(image)
    print(f'the image {count} min: {np.min(image)}')
    print(f'the image {count} max: {np.max(image)}')
    print(f'the image {count} mean: {np.mean(image)}')
    # print(f'the image {count} mean: {np.min(image)-np.mean(image)}')
    print(f'the dimension of {count} image: {image_np.shape}')
    min_value = np.min(image_np)
    max_value = np.max(image_np)
    image_np = (image_np - min_value) * (2.0) / (max_value - min_value) - 1.0

    # Perform slicing on the preprocessed array
    image_np_x_slice = slicing(image_np, no)
    image_np_x_slice1 = None

    if no == 128:
        # Set the desired scale factor for upsampling
        scale_factor = 2
        # Upsample the image using zoom
        image_np_x_slice1 = zoom(image_np_x_slice, (1, scale_factor, scale_factor, 1), order=3)
        image_np_x_slice1.shape
    else:
        image_np_x_slice1 = image_np_x_slice

    return image_np_x_slice1, count

In [5]:
def data_augmentation(images):
    '''data augmentaion is done by rotation and flipping'''

    #Create empty arrays to store rotation and flipped images
    rotated_90 = np.empty_like(images)
    rotated_180 = np.empty_like(images)
    rotated_270 = np.empty_like(images)
    flipped_horizontal = np.empty_like(images)
    flipped_vertical = np.empty_like(images)

    # Rotate each image in the ndarray
    for i in range(images.shape[0]):
        for j in range(images.shape[3]):
            rotated_90[i, :, :, j] = np.rot90(images[i, :, :, j], k=1)
            rotated_180[i, :, :, j] = np.rot90(images[i, :, :, j], k=2)
            rotated_270[i, :, :, j] = np.rot90(images[i, :, :, j], k=3)
            flipped_horizontal[i, :, :, j] = np.flip(images[i, :, :, j], axis=1)
            flipped_vertical[i, :, :, j] = np.flip(images[i, :, :, j], axis=0)


    # Concatenate the original, rotated, flipped horizontally, and flipped vertically images
    final_images = np.concatenate((images, rotated_90, rotated_180, rotated_270, flipped_horizontal, flipped_vertical), axis=0)

    return final_images

In [7]:
#training examples
# Folder path containing the images
folder_path = 'final_year_project/auto-encoder/dataset/ls'

# List all files in the folder
file_list = os.listdir(folder_path)
file_list[0], file_list[1] = file_list[1], file_list[0]
print(file_list)
# Iterate over the files and preprocess the images
count = 0
preprocessed_images = []
for file_name in file_list:
    file_path = os.path.join(folder_path, file_name)
    if os.path.isfile(file_path):
        image = imageio.imread(file_path)
        preprocessed_image, count = preprocess_image(image, 128, count)
        preprocessed_images.append(preprocessed_image)


# Convert the list of preprocessed images to a NumPy array
preprocessed_array = np.array(preprocessed_images)
preprocessed_array = preprocessed_array.reshape(preprocessed_array.shape[0], -1)

for i, j in enumerate(range(preprocessed_array.shape[0])):
    print(f"the ouput dimension for image {i}: {preprocessed_array[j][0].shape}")
    count = i

training_sample = np.concatenate((preprocessed_array[0][0], preprocessed_array[1][0]), axis=0)

#data augmentation
training_sample = data_augmentation(training_sample)
print(f'shape of training example: {training_sample.shape}')

['01jan19_merged_sample2_ls.tif', '01jan19_merged_sample1_ls.tif']
the image 1 min: 2.0
the image 1 max: 1311.0
the image 1 mean: 362.4297180175781
the dimension of 1 image: (567, 572, 4)
the image 2 min: 1.0
the image 2 max: 1337.0
the image 2 mean: 373.217041015625
the dimension of 2 image: (295, 569, 4)
the ouput dimension for image 0: (16, 256, 256, 4)
the ouput dimension for image 1: (8, 256, 256, 4)
shape of training example: (144, 256, 256, 4)


C:\Users\otiwa\AppData\Local\Temp\ipykernel_19252\3980498972.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  preprocessed_array = np.array(preprocessed_images)


In [8]:
training_sample.min()

-1.0043683

In [9]:
#testing examples
# Folder path containing the images
folder_path = 'final_year_project/auto-encoder/dataset/hs'

# List all files in the folder
file_list = os.listdir(folder_path)
print(file_list)
# Iterate over the files and preprocess the images
count = 0
preprocessed_images = []
for file_name in file_list:
    file_path = os.path.join(folder_path, file_name)
    if os.path.isfile(file_path):
        image = imageio.imread(file_path)
        preprocessed_image, count = preprocess_image(image, 256, count)
        preprocessed_images.append(preprocessed_image)


# Convert the list of preprocessed images to a NumPy array
preprocessed_array = np.array(preprocessed_images)
preprocessed_array = preprocessed_array.reshape(preprocessed_array.shape[0], -1)

for i, j in enumerate(range(preprocessed_array.shape[0])):
    print(f"the ouput dimension for image {i}: {preprocessed_array[j][0].shape}")
    count = i

testing_sample = np.concatenate((preprocessed_array[0][0], preprocessed_array[1][0]), axis=0)

#data_augmentation
testing_sample = data_augmentation(testing_sample)
print(f'shape of training example: {testing_sample.shape}')

['15jan19_merged_sample1_hs.tif', '15may19_merged_sample2_hs.tif']
the image 1 min: 1.0
the image 1 max: 266.0
the image 1 mean: 95.2601547241211
the dimension of 1 image: (589, 1138, 4)
the image 2 min: 24.0
the image 2 max: 598.0
the image 2 mean: 152.09535217285156
the dimension of 2 image: (1134, 1144, 4)
the ouput dimension for image 0: (8, 256, 256, 4)
the ouput dimension for image 1: (16, 256, 256, 4)
shape of training example: (144, 256, 256, 4)


C:\Users\otiwa\AppData\Local\Temp\ipykernel_19252\2535726390.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  preprocessed_array = np.array(preprocessed_images)


In [10]:
testing_sample.max()

1.0

antoencoder architecture

In [11]:
def auto_encoder(model, img_h=256, img_w=256, learning_rate=0.001):
    '''the auto encoder architecture consist of Encoder -> Latent space -> Decoder'''

    #encoder model
    input1 = Input(shape=(img_h, img_w, 4))
    encoder_output1 = Conv2D(16, (3, 3), activation='relu', padding='same', strides=2)(input1)
    encoder_output2 = Conv2D(32, (3, 3), activation='relu', padding='same', strides=2)(encoder_output1)
    encoder_output3 = Conv2D(32, (3, 3), activation='relu', padding='same', strides=2)(encoder_output2)
    encoder_output4 = Conv2D(32, (3, 3), activation='relu', padding='same', strides=2)(encoder_output3)
    encoder_output5 = Conv2D(32, (3, 3), activation='relu', padding='same', strides=2)(encoder_output4)
    encoder_output6 = Conv2D(32, (3, 3), activation='relu', padding='same', strides=2)(encoder_output5)

    #latent space
    decoder_output = Conv2D(128, (3, 3), activation='relu', padding='same', strides=2)(encoder_output6)

    #decoder model
    decoder_output1 = Conv2DTranspose(32, (3, 3), activation='relu', padding='same', strides=2)(decoder_output)
    decoder_output1 = Conv2DTranspose(32, (3, 3), activation='relu', padding='same', strides=2)(decoder_output1)
    decoder_output1 = BatchNormalization()(decoder_output1)
    decoder_output1_skip = Add()([encoder_output5, decoder_output1])

    decoder_output1 = Conv2DTranspose(64, (3, 3), activation='relu', padding='same', strides=2)(decoder_output1_skip)
    decoder_output1 = Conv2DTranspose(64, (3, 3), activation='relu', padding='same', strides=2)(decoder_output1)
    decoder_output2 = Conv2DTranspose(32, (3, 3), activation='relu', padding='same', strides=2)(decoder_output1)
    decoder_output2 = BatchNormalization()(decoder_output2)
    decoder_output2_skip = Add()([encoder_output2, decoder_output2])

    decoder_output3 = Conv2DTranspose(16, (3, 3), activation='relu', padding='same', strides=2)(decoder_output2_skip)
    decoder_output3 = BatchNormalization()(decoder_output3)
    decoder_output3_skip = Add()([encoder_output1, decoder_output3])

    decoder_output4 = Conv2DTranspose(4, (3, 3), activation='tanh', padding='same', strides=2)(decoder_output3_skip)
    decoder_output4 = BatchNormalization()(decoder_output4)
    decoder_output4_skip = Add()([input1, decoder_output4])
    # decoder_output5 = Conv2DTranspose(4, (3, 3), activation='relu', padding='same', strides=2)(decoder_output4_skip)

    # model definition
    model = Model(inputs=input1, outputs=decoder_output4_skip)
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['acc'])

    # summarize model
    print(model.summary())

    return model

In [12]:
# Create the model outside the TensorFlow function
model = auto_encoder(model= None, img_h=256, img_w=256)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 4  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 128, 128, 16  592         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 64, 64, 32)   4640        ['conv2d[0][0]']                 
                                                                                              

training the model

In [13]:
model.fit(training_sample, testing_sample, batch_size=1, epochs=50)

Epoch 1/50
144/144 [==============================] - 9s 38ms/step - loss: 0.4392 - acc: 0.6562
Epoch 2/50
144/144 [==============================] - 5s 36ms/step - loss: 0.0404 - acc: 0.8667
Epoch 3/50
144/144 [==============================] - 6s 39ms/step - loss: 0.0362 - acc: 0.8747
Epoch 4/50
144/144 [==============================] - 6s 40ms/step - loss: 0.0349 - acc: 0.8762
Epoch 5/50
144/144 [==============================] - 6s 40ms/step - loss: 0.0343 - acc: 0.8769
Epoch 6/50
144/144 [==============================] - 5s 38ms/step - loss: 0.0341 - acc: 0.8771
Epoch 7/50
144/144 [==============================] - 6s 40ms/step - loss: 0.0339 - acc: 0.8773
Epoch 8/50
144/144 [==============================] - 5s 37ms/step - loss: 0.0338 - acc: 0.8775
Epoch 9/50
144/144 [==============================] - 6s 39ms/step - loss: 0.0338 - acc: 0.8775
Epoch 10/50
144/144 [==============================] - 5s 38ms/step - loss: 0.0337 - acc: 0.8776
Epoch 11/50
144/144 [==================

In [14]:
# Save the model weights to the current directory
model.save_weights('autoencoder_weights_new.h5')

In [15]:
training_sample[1].shape

(256, 256, 4)

predicting the output

In [16]:
# Rebuild the autoencoder model
model = auto_encoder(model=None, img_h=256, img_w=256)

# Load the saved weights
model.load_weights('autoencoder_weights_new.h5')

# Generate predictions
predictions = model.predict(training_sample)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 256, 256, 4  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_7 (Conv2D)              (None, 128, 128, 16  592         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_8 (Conv2D)              (None, 64, 64, 32)   4640        ['conv2d_7[0][0]']               
                                                                                            

saving the ouput

In [17]:
# Convert the data type of the predictions array to a suitable format
# Modify this based on the requirements of your predicted output
# predictions = predictions.squeeze()  # Remove singleton dimensions if present
# predictions = np.uint8(predictions * 255)  # Scale values to 0-255 range and convert to uint8

# Create a directory to save the predictions if it doesn't exist
directory = r'final_year_project\auto-encoder1\dataset'
if not os.path.exists(directory):
    os.makedirs(directory)
for i in range(predictions.shape[0]):
    # Save the predicted output as a TIFF file
    output_path = os.path.join(directory, f'predicted{[i]}.tif')
    tifffile.imwrite(output_path, predictions[i], planarconfig='contig')

# Display a download link
print(f"Download the predicted output: [predicted_output.tif]({output_path})")

Download the predicted output: [predicted_output.tif](final_year_project\auto-encoder1\dataset\predicted[143].tif)


In [18]:
# Create a directory to save the predictions if it doesn't exist
directory = r'final_year_project\auto-encoder3\dataset'
if not os.path.exists(directory):
    os.makedirs(directory)
for i in range(predictions.shape[0]):
    # Save the predicted output as a TIFF file
    output_path = os.path.join(directory, f'real_input{[i]}.tif')
    tifffile.imwrite(output_path, training_sample[i], planarconfig='contig')

# Display a download link
print(f"Download the predicted output: [predicted_output.tif]({output_path})")

Download the predicted output: [predicted_output.tif](final_year_project\auto-encoder3\dataset\real_input[143].tif)


In [19]:
# Create a directory to save the predictions if it doesn't exist
directory = r'final_year_project\auto-encoder2\dataset'
if not os.path.exists(directory):
    os.makedirs(directory)
for i in range(predictions.shape[0]):
    # Save the predicted output as a TIFF file
    output_path = os.path.join(directory, f'real_output{[i]}.tif')
    tifffile.imwrite(output_path, testing_sample[i], planarconfig='contig')

# Display a download link
print(f"Download the predicted output: [predicted_output.tif]({output_path})")

Download the predicted output: [predicted_output.tif](final_year_project\auto-encoder2\dataset\real_output[143].tif)


In [ ]:
import shutil

shutil.rmtree(r'final_year_project\auto-encoder2\dataset')

Peak signal to noise ratio

In [20]:
def calculate_psnr(original, predicted):
    mse = np.mean((original - predicted) ** 2)
    max_value = np.max(original)
    psnr = 20 * math.log10(max_value / math.sqrt(mse))
    return psnr

In [21]:
#original psnr
#psnr when calculated with the orignal and the ground truth
psnr = 0
for i in range(training_sample.shape[0]):
    psnr += calculate_psnr(training_sample[i], testing_sample[i])
    psnr /= training_sample.shape[0]
print(f'PSNR in terms of training:{psnr}')

PSNR in terms of training:0.09003838477133778


In [22]:
#predcition psnr
#psnr when calculated with the prediction and the ground truth
psnr = 0
for i in range(training_sample.shape[0]):
    psnr += calculate_psnr(predictions[i], testing_sample[i])
    psnr /= training_sample.shape[0]
print(f'PSNR in terms of predciton:{psnr}')

PSNR in terms of predciton:0.023833554410302035
